# 贝叶斯向量自回归

**发布日期**: 2019年10月15日

**文章作者**: Xinyu Chen (陈新宇) [[GitHub主页](https://github.com/xinychen)]

**下载**: 本Jupyter Notebook可在GitHub仓库[GraphicalML](https://github.com/mobility-computing/GrapicalML/blob/master/content/bvar.ipynb)中进行下载和使用。


## 0 关于本文

- 讨论贝叶斯向量自回归模型的形式
- 介绍如何实现贝叶斯向量自回归
- 分析贝叶斯向量自回归的应用

## 1 向量自回归模型

向量自回归模型的英文名称为Vector Autoregressive model，常被简写成VAR。向量自回归的出现由来已久，可以追溯到上个世纪，人们构建向量自回归模型主要出于以下考虑：

- 时间序列分析从单一时间序列 (time series data) 拓展到了多元时间序列 (multivariate time series)，任意时刻$t$下，观测样本从1变成了$N$，其中，$N$表示多元时间序列中时间序列的数量。
- 标准的自回归模型 (Autoregressive model, 简称AR) 形式过于简单，无法很好地在多元时间序列分析中发挥作用。

为方便理解，不妨先看一下标准的自回归模型：给定单一时间序列$\boldsymbol{y}\in\mathbb{R}^{T}$，其长度为$T$，则对于任意时间点$t$，有
\begin{equation}
y_{t}=\sum_{k=1}^{d}a_ky_{t-k},
\end{equation}
其中，常数$d$表示自回归模型的阶数，也可以将$d$简单地理解成当前时间点关联过去时间点的数量。
